In [1]:
from music21 import *
import numpy as np
from hmmlearn.hmm import MultinomialHMM

In [2]:
# Analyze Bach music dataset
all_bach_paths = corpus.getComposer('bach')
print("Total number of Bach pieces to process from music21:", len(all_bach_paths))

parsed_bach = []
keys = []
songs_by_key = {}
for i, p_bach in enumerate(all_bach_paths):
    p = corpus.parse(p_bach)
    if len(p.parts) != 4:
        continue
    parsed_bach.append(p)
    if p.analyze('key') not in keys:
        keys.append(p.analyze('key'))
        songs_by_key[p.analyze('key')] = [p]
    else:
        songs_by_key[p.analyze('key')].append(p)
        
print("Updated number of Bach pieces to process from music21:", len(parsed_bach))
for key in keys:
    print("Number of songs in", key, "key:", len(songs_by_key[key]))

Total number of Bach pieces to process from music21: 433
Updated number of Bach pieces to process from music21: 368
Number of songs in g minor key: 43
Number of songs in d minor key: 29
Number of songs in c minor key: 10
Number of songs in b minor key: 27
Number of songs in A major key: 32
Number of songs in D major key: 25
Number of songs in a minor key: 47
Number of songs in G major key: 45
Number of songs in C major key: 14
Number of songs in e minor key: 19
Number of songs in F major key: 25
Number of songs in B- major key: 22
Number of songs in E major key: 9
Number of songs in E- major key: 9
Number of songs in f# minor key: 7
Number of songs in f minor key: 2
Number of songs in A- major key: 1
Number of songs in b- minor key: 2


In [57]:
# Transpose all songs into C
transposed_bach_major = []
transposed_bach_minor = []
for p in parsed_bach:
    k = p.analyze('key')
    i = interval.Interval(k.tonic, pitch.Pitch('C'))
    if 'major' in k.name:
        transposed_bach_major.append(p.transpose(i))
    elif 'minor' in k.name:
        transposed_bach_minor.append(p.transpose(i))

In [40]:
class Song:
    def __init__(self):
        """Initialize Song class.
        
        Attributes:
            score (Score): Song from music21 Score class
            key (str): Key of the song
            parts (ndarray(p,n)): Array of parts(p) and notes(n)
            n_parts (int): Number of parts in the song
            stream (Stream): music21 Stream object generated from parts
        """
        return
        
    def parse(self, score):
        """Parse the MusicXML score into a trainable format"""
        parts = []
        for i, p in enumerate(score.parts):
            # Get notes and rests for part i
            notes = []
            for n in p.recurse().notesAndRests:
                if n.isRest:
                    notes.append((0,float(n.quarterLength)))
                else:
                    notes.append((n.pitch,float(n.quarterLength)))
                
            # Add notes list for each part to part list
            parts.append(notes)  
            
            self.score = score
            self.key = score.analyze('key')
            self.parts = np.array(parts, dtype=object)
            self.n_parts = len(parts)  
        
    def gen_stream(self, parts=None):
        """Generate a song from the parts and notes parsed
        
        Parameters:
            parts (list): list of part indices to generate
        """
        # If None, generate all parts
        if parts is None:
            parts = np.arange(self.n_parts)
        parts = np.array(parts, dtype=int)
        
        # Generate the song for the parts indicated
        s = stream.Stream()
        for i, notes in enumerate(self.parts[parts]):
            p = stream.Part(id=i)
            for n in notes: 
                if n[0] == 0:
                    p.append(note.Rest(quarterLength=n[1]))
                else:
                    p.append(note.Note(n[0],quarterLength=n[1]))
            s.insert(0,p)
        self.stream = s
        
    def play(self):
        """Play the song generated by gen_stream"""
        self.stream.show('midi')  
        
    def save(self, filename):
        """Save the generated song as a midi file"""
        self.stream.write('midi')

In [53]:
class MusicHMM:
    def __init__(self, n_components):
        """Initialize MusicHMM class
        
        Attributes:
            songs (list): List of Song class objects to train on
            n_components (int): number of components for the HMM
            HMMs (list(MultinomialHMM)): HMM objects for each part
            notes (ndarray): Array of possible states
            note_ind (func): Map from note state to index
            obs (list): Observed sequences of notes by index
            obs_len (list): length of observation sequence for each song
        """
        self.n_components = n_components
        
        
    def fit(self, songs, parts=None):
        """Train the HMM on the list of songs for part indices given
        
        Parameters:
            songs (list): List of parsed Song objects to train on
            parts (list): List of part indices to train
        """
        self.songs = songs
        
        # If None, generate all parts
        if parts is None:
            parts = np.arange(songs[0].n_parts)
        parts = np.array(parts, dtype=int)
        
        # Generate state space data and observations by index
        self.init_states(parts)
        self.init_obs_matrices(parts)
        
        # Train a HMM for each part (???)
        n_parts = len(parts)
        HMMs = []
        for i in range(n_parts):
            obs = np.array(self.obs[i]).reshape(-1, 1)   # Reshape observations
            hmm = MultinomialHMM(n_components=self.n_components)
            hmm.fit(obs, lengths=self.obs_len[i])
            HMMs.append(hmm)
            
        self.HMMs = HMMs
        
        
    def init_states(self, parts):
        """Create and save a dictionary of unique notes"""
        notes = []
        for song in self.songs:
            for i, p in enumerate(song.parts[parts]):
                for n in p:
                    if n not in notes:
                        notes.append(n)
                    
        self.notes = np.array(notes)
        self.note_ind = lambda n: notes.index(n)
        
        
    def init_obs_matrices(self, parts):
        """Create the matrices of note indices observed"""
        # Initialize
        n_parts = len(parts)
        obs = [[]] * n_parts
        obs_len = [[]] * n_parts
        
        for i in range(n_parts):
            for song in self.songs:
                p = song.parts[parts[i]]
                seq = [self.note_ind(n) for n in p]
                obs[i] += seq
                obs_len[i].append(len(seq))
                
        self.obs = obs
        self.obs_len = obs_len 
                
               
    def gen_song(self, num_notes=40, parts=None):
        """Sample a new song from the HMM"""
        # If None, generate all parts
        if parts is None:
            parts = np.arange(self.songs[0].n_parts)
        parts = np.array(parts, dtype=int)
        
        s = stream.Score()
        for i in parts:
            hmm = self.HMMs[i]
            ind = hmm.sample(num_notes)[0].ravel()
            notes = self.notes[ind]
            
            p = stream.Part(id=i)
            for n in notes: 
                if n[0] == 0:
                    p.append(note.Rest(quarterLength=n[1]))
                else:
                    p.append(note.Note(n[0],quarterLength=n[1]))
            s.insert(0,p)
            
        new_song = Song()
        new_song.parse(s)
        new_song.gen_stream()
        
        return new_song

In [36]:
# Test on one song
song = Song()
song.parse(parsed_bach[1])
song.gen_stream()
song.play()

In [38]:
# Train on one song
mhmm = MusicHMM(2)
mhmm.fit([song], parts=[0,1,2,3])
new_song = mhmm.gen_song()
new_song.play()

In [58]:
# Create song classes for all transposed Bach music
maj_songlist = []
min_songlist = []

for p in transposed_bach_major:
    song = Song()
    song.parse(p)
    maj_songlist.append(song)

for p in transposed_bach_minor:
    song = Song()
    song.parse(p)
    min_songlist.append(song)

In [62]:
# Train on songlist of transposed music
mhmm = MusicHMM(5)
mhmm.fit(maj_songlist)

In [63]:
new_song = mhmm.gen_song(num_notes=20, parts=None)
new_song.play()